In [1]:
import pandas as pd

In [2]:
import pandas as pd
#Reading bit by bit since data is too large 
chunk_size = 500000
# Initialize an empty list to collect chunks
chunks = []
for chunk in pd.read_csv('eda.csv', chunksize=chunk_size):
    chunks.append(chunk)
#Convert to dataframe
df = pd.concat(chunks, ignore_index=True)

In [3]:
df.head()

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,...,tip_amount_transformed,pickup_week,pickup_date,day_of_week,hour,month,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude
0,2020-01-01 00:28:15,2020-01-01 00:33:03,1.0,0.422371,238,239,1.0,0.237510,3.0,0.5,...,0.410110,2019-12-30/2020-01-05,2020-01-01,Wednesday,0,1,40.791705,-73.973049,40.783961,-73.978632
1,2020-01-01 00:35:39,2020-01-01 00:43:04,1.0,0.422371,239,238,1.0,0.277161,3.0,0.5,...,0.433048,2019-12-30/2020-01-05,2020-01-01,Wednesday,0,1,40.783961,-73.978632,40.791705,-73.973049
2,2020-01-01 00:47:41,2020-01-01 00:53:52,1.0,0.322204,238,238,1.0,0.237510,3.0,0.5,...,0.000000,2019-12-30/2020-01-05,2020-01-01,Wednesday,0,1,40.791705,-73.973049,40.791705,-73.973049
3,2020-01-01 00:55:23,2020-01-01 01:00:14,1.0,0.355593,238,151,1.0,0.217684,0.5,0.5,...,0.323179,2019-12-30/2020-01-05,2020-01-01,Wednesday,0,1,40.791705,-73.973049,40.797962,-73.968168
4,2020-01-01 00:01:58,2020-01-01 00:04:16,1.0,0.222037,193,193,2.0,0.138382,0.5,0.5,...,0.000000,2019-12-30/2020-01-05,2020-01-01,Wednesday,0,1,40.760314,-73.941997,40.760314,-73.941997


In [4]:
df.shape

(5321529, 51)

In [5]:
df1 = df

In [6]:
df1['tpep_pickup_datetime'] = pd.to_datetime(df1['tpep_pickup_datetime'])
df1['tpep_dropoff_datetime'] = pd.to_datetime(df1['tpep_dropoff_datetime'])

In [7]:
columns_to_drop = ['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'trip_type', 'pickup_week', 'pickup_date', 'day_of_week']
df1 = df1.drop(columns=columns_to_drop)

In [8]:
df1 = df1.apply(pd.to_numeric, errors='coerce')

In [9]:
df1.to_csv('eda1.csv', index = False)

In [10]:
from sklearn.preprocessing import StandardScaler
chunk_size = 500000
numerical_cols = [
    'trip_distance', 'fare_amount', 'extra', 'mta_tax', 'tip_amount',
    'tolls_amount', 'improvement_surcharge', 'congestion_surcharge', 
    'fare_per_mile', 'avg_fare_per_pulocation', 'avg_fare_per_dolocation', 
    'trip_duration', 'log_trip_distance', 'log_trip_duration', 
    'log_fare_amount', 'log_tip_amount', 'boxcox_trip_distance', 
    'boxcox_trip_duration', 'boxcox_fare_amount', 'boxcox_tip_amount',
    'pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude'
]

scaler = StandardScaler()

In [11]:
for chunk in pd.read_csv('eda1.csv', usecols=numerical_cols, chunksize=chunk_size):
    scaler.partial_fit(chunk)

In [12]:
scaled_chunks = []
for chunk in pd.read_csv('eda1.csv', chunksize=chunk_size):
    chunk[numerical_cols] = scaler.transform(chunk[numerical_cols])
    scaled_chunks.append(chunk)

In [13]:
#Now we save the scaled values into a dataframe for future use

df2 = pd.concat(scaled_chunks, ignore_index=True)

In [14]:
df2.head()

,passenger_count,trip_distance,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,...,boxcox_fare_amount,boxcox_tip_amount,tip_given,tip_amount_transformed,hour,month,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude
0,1.0,-0.446200,238,239,1.0,-0.767335,1.527338,0.038378,-0.141140,-0.032898,...,-0.767968,0.007112,1,0.410110,0,1,1.551130,0.252161,1.121102,-0.069330
1,1.0,-0.446200,239,238,1.0,-0.499769,1.527338,0.038378,-0.122839,-0.032898,...,-0.395814,0.029080,1,0.433048,0,1,1.212009,-0.012898,1.441136,0.187912
2,1.0,-1.057243,238,238,1.0,-0.767335,1.527338,0.038378,-0.427856,-0.032898,...,-0.767968,-0.362171,1,0.000000,0,1,1.551130,0.252161,1.441136,0.187912
3,1.0,-0.853562,238,151,1.0,-0.901117,-0.502998,0.038378,-0.208244,-0.032898,...,-0.970941,-0.074944,1,0.323179,0,1,1.551130,0.252161,1.699736,0.412787
4,1.0,-1.668286,193,193,2.0,-1.436249,-0.502998,0.038378,-1.037889,-0.032898,...,-1.917596,-1.397853,0,0.000000,0,1,0.176372,1.726368,0.143756,1.618643


### Here optionally if needed we can save the scaled down csv, but for now we will not be doing that

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
#setting variables

train_size = 0.6
validation_size = 0.2
test_size = 0.2

In [17]:
#Lists to hold the splitted data

train_chunks = []
validation_chunks = []
test_chunks = []

In [18]:
#Forecfully have to save the scaled file since I failed to use the dataframe for the train test split :(

df2.to_csv('scaled_eda1.csv', index = False)

In [19]:
for chunk in pd.read_csv('scaled_eda1.csv', chunksize=chunk_size):
    train, temp = train_test_split(chunk, test_size=(1 - train_size))
    validation, test = train_test_split(temp, test_size=(test_size / (test_size + validation_size)))
    
    train_chunks.append(train)
    validation_chunks.append(validation)
    test_chunks.append(test)

# Individual dataframes
train_df = pd.concat(train_chunks, ignore_index=True)
validation_df = pd.concat(validation_chunks, ignore_index=True)
test_df = pd.concat(test_chunks, ignore_index=True)

# Separate csv files for idk maybe useful
train_df.to_csv('train1.csv', index=False)
validation_df.to_csv('validation1.csv', index=False)
test_df.to_csv('test1.csv', index=False)

In [20]:
print(f'The training dataset is : {train_df.shape}')
print(f'The validation dataset is : {validation_df.shape}')
print(f'The testing dataset is : {test_df.shape}')

The training dataset is : (3192917, 45)
The validation dataset is : (1064306, 45)
The testing dataset is : (1064306, 45)


### Using a simple model to get a performance benchmark here we are taking Linear Regression 

In [21]:
from sklearn.linear_model import LinearRegression 

In [22]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [23]:
# train_df['tpep_pickup_datetime'] = pd.to_datetime(train_df['tpep_pickup_datetime'])
# train_df['tpep_dropoff_datetime'] = pd.to_datetime(train_df['tpep_dropoff_datetime'])

In [24]:
# columns_to_drop = ['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'trip_type', 'pickup_week', 'pickup_date', 'day_of_week']
# train_df = train_df.drop(columns=columns_to_drop)

In [25]:
# train_df = train_df.apply(pd.to_numeric, errors='coerce')

In [26]:
train_df.dtypes

passenger_count             float64
trip_distance               float64
PULocationID                  int64
DOLocationID                  int64
payment_type                float64
fare_amount                 float64
extra                       float64
mta_tax                     float64
tip_amount                  float64
tolls_amount                float64
improvement_surcharge       float64
congestion_surcharge        float64
tip_percentage              float64
fare_per_mile               float64
avg_fare_per_pulocation     float64
avg_fare_per_dolocation     float64
ratecode_negotiated_fare       bool
ratecode_group_ride            bool
ratecode_shared_ride           bool
ratecode_other                 bool
ratecode_special_rates         bool
pickup_hour                   int64
pickup_day                    int64
pickup_month                  int64
pickup_day_of_week            int64
is_peak_hour                  int64
is_weekend                    int64
same_zone                   

In [28]:
linear_model = LinearRegression()

# Train the model on the training data
linear_model.fit(train_df.drop('fare_amount', axis=1), train_df['fare_amount'])

# Predict on the validation data
val_predictions = linear_model.predict(validation_df.drop('fare_amount', axis=1))

# Evaluate the model's performance
mae = mean_absolute_error(validation_df['fare_amount'], val_predictions)
mse = mean_squared_error(validation_df['fare_amount'], val_predictions)
rmse = mse ** 0.5
r2 = r2_score(validation_df['fare_amount'], val_predictions)

print(f"Linear Regression Performance on Validation Set:")
print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"R-squared: {r2}")

Linear Regression Performance on Validation Set:
MAE: 0.006030877428177153
MSE: 8.82999883504051e-05
RMSE: 0.00939680734879699
R-squared: 0.9999115445898294
